# Converting .fsm files to .txt files + metadata .json files

In [1]:
import numpy as np
import pandas as pd
import os

import collections.abc
collections.Iterable = collections.abc.Iterable
from specio import specread

import json

## From .fsm to .txt

In [2]:
def fsm_to_txt(in_filename, out_filename):
    spectra = specread("{}.fsm".format(in_filename))    # Loads the .fsm file
    
    x_coords = np.zeros(np.shape(spectra.amplitudes)[0])  # Placeholders for the ccordinates
    y_coords = np.zeros(np.shape(spectra.amplitudes)[0])
    seged = np.zeros(spectra.meta["n_x"])                 # A helper variable
    
    for i in range(spectra.meta["n_y"]): # Generate Y coordinates from the metadata (grid-like)
        y_coords[i * spectra.meta["n_x"] :(i + 1) * spectra.meta["n_x"]] = spectra.meta["y_init"] + i * spectra.meta["y_delta"]
    for i in range(len(seged)): # An array containing all X coordinates
        seged[i] = spectra.meta["x_init"] + i * spectra.meta["x_delta"]
    for i in range(spectra.meta["n_y"]): # Assigning the seged vector to every Y coordinate (rectangular measurement area)
        x_coords[i * spectra.meta["n_x"]:(i + 1) * spectra.meta["n_x"]] = seged
        
    df1 = pd.DataFrame({"X": x_coords, "Y" : y_coords}) # Computed coordinates
    df2 = pd.DataFrame(data = spectra.amplitudes, columns = spectra.wavelength) # Spectral values
    df = df1.join(df2)
    #Gives back the .fsm order.
    df = df.sort_values(by = ["Y", "X"], ascending = [False, True], axis = 0).reset_index(drop = True)
    df.to_csv("{}.txt".format(out_filename))
    return df

In [27]:
fsm_to_txt('/home/borbende/wsi/tma5_data/20221130/tma5_B1_res8_co16_2.2int_25', '/home/borbende/wsi/tma5_data/spectra/tma5_B1_res8_co16_2.2int_25')

,X,Y,4000.0,3996.0,3992.0,3988.0,3984.0,3980.0,3976.0,3972.0,...,684.0,680.0,676.0,672.0,668.0,664.0,660.0,656.0,652.0,648.0
0,-1087.499957,1087.499957,77.411308,77.460464,77.535202,77.504234,77.372345,77.254883,77.293266,77.307129,...,74.459511,73.175514,73.116592,72.993340,71.496033,71.617325,72.326767,69.526741,66.520111,69.082916
1,-1062.499958,1087.499957,77.289337,77.399300,77.389603,77.226494,77.128662,77.146088,77.254486,77.347115,...,74.033867,73.193382,71.775818,70.595383,70.456764,73.889084,75.923790,71.984230,68.013527,71.281479
2,-1037.499959,1087.499957,76.561172,76.483498,76.610512,76.699615,76.633003,76.591949,76.567276,76.519203,...,74.209076,74.265945,74.403267,74.896591,71.598083,70.167931,69.913086,67.942825,65.531174,65.730698
3,-1012.499960,1087.499957,77.460823,77.352539,77.254234,77.166367,77.044670,77.080040,77.334595,77.469147,...,72.680237,72.471077,75.340904,78.338089,75.780693,71.379814,70.368416,69.619019,71.857811,75.412712
4,-987.499961,1087.499957,77.274475,77.382568,77.274178,77.064880,77.034164,77.113701,77.146645,77.065453,...,76.378311,74.378792,73.446381,72.592712,71.527878,72.095634,70.374077,66.807022,63.663517,66.390007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7739,987.499961,-1087.499957,76.593880,76.592445,76.549217,76.583168,76.633072,76.724556,76.764137,76.664169,...,75.495277,74.096527,75.483742,79.630737,81.296989,77.406273,73.719765,69.559799,70.398270,79.592422
7740,1012.499960,-1087.499957,76.320404,76.418411,76.446014,76.437660,76.429855,76.411110,76.424530,76.460182,...,76.243011,74.977959,75.129974,74.064140,67.967667,65.049248,65.861259,67.542831,72.210854,75.004456
7741,1037.499959,-1087.499957,76.733429,76.703285,76.636627,76.571297,76.536743,76.538605,76.646912,76.668526,...,75.932411,75.525368,74.833191,74.146080,74.538063,74.210892,71.196136,68.196419,68.629730,70.788292
7742,1062.499958,-1087.499957,73.095757,73.185905,73.042793,72.927498,73.010269,73.074844,73.150047,73.180023,...,73.361237,73.076866,73.113327,74.444458,73.888741,70.036018,68.109077,67.479202,67.097725,67.491386


In [5]:
%%time

in_folder = '/home/borbende/wsi/tma5_data/20221206'  # Linux convention
#in_folder = r"D:\ELTE\kutatas2\csongor_2025_09_30"  # Windows convention
out_folder = '/home/borbende/wsi/tma5_data/spectra'
#out_folder = f"{in_folder}\spectra"

# Check if the folder exists, if not, create it
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

files = [f[:-4] for f in os.listdir(in_folder) if f.endswith(".fsm")] # Find .fsm files in "in_folder"

for i in files:
    fsm_to_txt(f'{in_folder}/{i}', f'{out_folder}/{i}')

CPU times: total: 15.3 s
Wall time: 17.1 s


In [6]:
files # Some example .fsm files

['CStmacrc_12mic_A1_res8_co16_2.2int_50',
 'CStmacrc_12mic_A2_res8_co16_2.2int_50',
 'CStmacrc_12mic_B3_res8_co16_2.2int_50',
 'CStmacrc_16mic_A1_res8_co16_2.2int_50',
 'CStmacrc_16mic_B3_res8_co16_2.2int_50',
 'CStmacrc_16mic_C3_res8_co16_2.2int_50',
 'CStmacrc_2mic_A1_res8_co16_2.2int_50',
 'CStmacrc_2mic_B3_res8_co16_2.2int_50',
 'CStmacrc_2mic_C3_res8_co16_2.2int_50',
 'CStmacrc_8mic_A1_res8_co16_2.2int_50',
 'CStmacrc_8mic_B3_res8_co16_2.2int_50',
 'CStmacrc_8mic_C3_res8_co16_2.2int_50']

In [7]:
pd.read_csv(f"{out_folder}\\{files[0]}.txt") # The result .txt reloaded

,Unnamed: 0,X,Y,4000.0,3996.0,3992.0,3988.0,3984.0,3980.0,3976.0,...,784.0,780.0,776.0,772.0,768.0,764.0,760.0,756.0,752.0,748.0
0,0,-1074.999957,1074.999957,88.751510,88.866585,88.704990,88.531395,88.611946,88.820420,88.963110,...,88.073250,88.073250,88.137660,88.177025,88.194170,88.18522,88.167720,88.25104,88.362206,88.388560
1,1,-1024.999959,1074.999957,83.406500,83.446400,83.370130,83.257860,83.267230,83.348700,83.427630,...,84.488330,84.565704,84.655525,84.682144,84.721930,84.76389,84.734940,84.77264,84.915670,84.991660
2,2,-974.999961,1074.999957,86.729290,86.868990,86.814865,86.748090,86.749565,86.851210,86.969830,...,85.781456,85.844635,85.835060,85.795900,85.860780,85.97002,86.033936,86.08711,86.148330,86.223465
3,3,-924.999963,1074.999957,85.454040,85.528950,85.493520,85.331375,85.244360,85.328995,85.449875,...,85.202710,85.155930,85.145905,85.177770,85.252650,85.35329,85.393555,85.37140,85.396630,85.465300
4,4,-874.999965,1074.999957,88.536705,88.610090,88.560150,88.374310,88.275750,88.424065,88.548676,...,87.653610,87.716520,87.678090,87.662710,87.698190,87.74785,87.818060,87.91302,87.959550,87.955986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1931,1931,874.999965,-1074.999957,83.245170,83.289220,83.354650,83.414154,83.406265,83.447105,83.465110,...,84.639145,84.734690,84.778656,84.811240,84.881874,84.92048,84.845440,84.79075,84.871730,85.016335
1932,1932,924.999963,-1074.999957,85.966866,86.024890,85.932520,85.942690,86.077965,86.152590,86.152540,...,86.584060,86.633420,86.705460,86.744415,86.776530,86.73698,86.638970,86.63539,86.729240,86.845810
1933,1933,974.999961,-1074.999957,82.208750,82.227510,82.183320,82.157090,82.149160,82.172040,82.256775,...,84.378944,84.441290,84.382680,84.332405,84.380520,84.48322,84.542656,84.58022,84.649086,84.737120
1934,1934,1024.999959,-1074.999957,82.791580,82.922676,83.012375,82.930080,82.855920,82.968056,83.167244,...,83.769800,83.817570,83.877975,83.965996,84.001000,83.95451,83.893350,83.93394,84.044464,84.061840


## Metadata extraction

In [2]:
def meta_to_json(in_filename, out_filename):
    spectra = specread("{}.fsm".format(in_filename)) # Loads .fsm file
    meta = spectra.meta.copy()      # Accessing the metadata
    meta["signature"] = meta["signature"].decode("utf-8") # 1 byte was left out in this entry
    with open("{}.json".format(out_filename), "w") as f: # save as json
        json.dump(meta, f, indent=2)
    return 0

In [13]:
%%time

in_folder = '/home/borbende/wsi/tma5_data/20221206'
#in_folder = r"D:\ELTE\kutatas2\20221130\20221130"
out_folder = '/home/borbende/wsi/tma5_data/metadata'
#out_folder = f"{in_folder}\metadata"

# Check if the folder exists, if not, create it
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

files = [f[:-4] for f in os.listdir(in_folder) if f.endswith(".fsm")]

for i in files:
    meta_to_json(f'{in_folder}/{i}', f'{out_folder}/{i}')

CPU times: total: 672 ms
Wall time: 2.93 s
